In [ ]:
#pip install yfinance
#pip install PyPortfolioOpt

In [101]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import pypfopt
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

### Stock Selection

In [84]:
# Stock Selection
stock_list = ["D05.SI","Z74.SI","J36.SI","O39.SI","U11.SI","F34.SI","Y92.SI","Q0F.SI","BN4.SI","C38U.SI","C07.SI"]
#9CI.SI

In [85]:
START = '2015-01-01'
END = '2022-12-31'
df = pd.DataFrame()
period = "1y"
for company in stock_list:
    stock = yf.Ticker(company)
    hist = stock.history(start=START, end=END).Close
    df[company] = hist
df = df.dropna()
df

,D05.SI,Z74.SI,J36.SI,O39.SI,U11.SI,F34.SI,Y92.SI,Q0F.SI,BN4.SI,C38U.SI,C07.SI
Date,,,,,,,,,,,
2015-01-02 00:00:00+08:00,13.822886,2.709047,45.657070,7.215527,17.721083,2.431841,0.002571,1.414135,6.068272,1.303257,30.024014
2015-01-05 00:00:00+08:00,13.512717,2.702083,45.305973,7.112447,17.323423,2.402094,0.002516,1.402575,5.951838,1.309645,29.306082
2015-01-06 00:00:00+08:00,13.371112,2.674226,44.693432,6.981881,16.860697,2.372347,0.002516,1.379456,5.698422,1.290480,28.645582
2015-01-07 00:00:00+08:00,13.350887,2.688154,44.327393,6.995624,16.947458,2.357473,0.002607,1.379456,5.718971,1.328811,28.523533
2015-01-08 00:00:00+08:00,13.539686,2.743867,44.484264,7.215527,17.366802,2.409531,0.002607,1.398722,5.801159,1.296869,28.645582
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23 00:00:00+08:00,32.153217,2.490043,48.404598,11.612528,29.245790,3.981184,0.661024,1.662064,7.003848,1.916278,27.775372
2022-12-27 00:00:00+08:00,32.398804,2.499732,48.872322,11.574977,29.274183,3.981184,0.661024,1.670952,6.984816,1.916278,27.794725
2022-12-28 00:00:00+08:00,32.304352,2.499732,48.614597,11.593753,29.302576,4.038605,0.665849,1.653176,6.946752,1.916278,27.475357


### Weights Method

In [86]:
mu = mean_historical_return(df)
S = CovarianceShrinkage(df).ledoit_wolf()

ef = EfficientFrontier(mu, S)

weights = ef.max_sharpe()

cleaned_weights = ef.clean_weights()
print(cleaned_weights)

ef.portfolio_performance(verbose=True)


OrderedDict([('D05.SI', 0.42802), ('Z74.SI', 0.0), ('J36.SI', 0.0), ('O39.SI', 0.01121), ('U11.SI', 0.02957), ('F34.SI', 0.10248), ('Y92.SI', 0.42872), ('Q0F.SI', 0.0), ('BN4.SI', 0.0), ('C38U.SI', 0.0), ('C07.SI', 0.0)])
Expected annual return: 48.8%
Annual volatility: 42.9%
Sharpe Ratio: 1.09


(0.4879940596406088, 0.429105918437198, 1.0906259725921312)

In [87]:
stock_weights = list(cleaned_weights.values())
print(stock_weights)

[0.42802, 0.0, 0.0, 0.01121, 0.02957, 0.10248, 0.42872, 0.0, 0.0, 0.0, 0.0]


In [88]:
log_returns = np.log(df/df.shift(1))
log_returns

,D05.SI,Z74.SI,J36.SI,O39.SI,U11.SI,F34.SI,Y92.SI,Q0F.SI,BN4.SI,C38U.SI,C07.SI
Date,,,,,,,,,,,
2015-01-02 00:00:00+08:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05 00:00:00+08:00,-0.022694,-0.002574,-0.007720,-0.014389,-0.022696,-0.012308,-0.021661,-0.008208,-0.019374,0.004890,-0.024202
2015-01-06 00:00:00+08:00,-0.010535,-0.010363,-0.013612,-0.018528,-0.027074,-0.012461,0.000000,-0.016621,-0.043511,-0.014742,-0.022796
2015-01-07 00:00:00+08:00,-0.001514,0.005195,-0.008224,0.001966,0.005133,-0.006290,0.035846,0.000000,0.003599,0.029271,-0.004270
2015-01-08 00:00:00+08:00,0.014042,0.020514,0.003533,0.030950,0.024443,0.021842,0.000000,0.013870,0.014269,-0.024332,0.004270
...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23 00:00:00+08:00,-0.006734,-0.007752,0.002567,0.004862,-0.008379,-0.002401,0.007326,0.027102,-0.005420,0.000000,-0.002784
2022-12-27 00:00:00+08:00,0.007609,0.003883,0.009616,-0.003239,0.000970,0.000000,0.000000,0.005333,-0.002721,0.000000,0.000697
2022-12-28 00:00:00+08:00,-0.002920,0.000000,-0.005287,0.001621,0.000969,0.014320,0.007273,-0.010695,-0.005465,0.000000,-0.011557


In [89]:
def calculate_returns(weights, returns):
    return np.sum(returns.mean() * weights) * 252

In [90]:
calculate_returns(stock_weights, log_returns)

0.3524224399125339

In [91]:
log_returns_cov = log_returns.cov()
log_returns_cov

,D05.SI,Z74.SI,J36.SI,O39.SI,U11.SI,F34.SI,Y92.SI,Q0F.SI,BN4.SI,C38U.SI,C07.SI
D05.SI,0.000150,0.000068,0.000050,0.000107,0.000112,0.000069,0.000075,0.000025,0.000092,0.000059,0.000085
Z74.SI,0.000068,0.000131,0.000036,0.000064,0.000064,0.000059,0.000053,0.000019,0.000069,0.000057,0.000070
J36.SI,0.000050,0.000036,0.000266,0.000048,0.000053,0.000044,0.000051,0.000021,0.000051,0.000045,0.000106
O39.SI,0.000107,0.000064,0.000048,0.000120,0.000103,0.000073,0.000050,0.000024,0.000092,0.000060,0.000085
U11.SI,0.000112,0.000064,0.000053,0.000103,0.000131,0.000068,0.000055,0.000020,0.000092,0.000056,0.000086
F34.SI,0.000069,0.000059,0.000044,0.000073,0.000068,0.000208,0.000043,0.000020,0.000088,0.000060,0.000079
Y92.SI,0.000075,0.000053,0.000051,0.000050,0.000055,0.000043,0.002011,0.000031,0.000078,0.000072,0.000084
Q0F.SI,0.000025,0.000019,0.000021,0.000024,0.000020,0.000020,0.000031,0.000207,0.000030,0.000018,0.000023
BN4.SI,0.000092,0.000069,0.000051,0.000092,0.000092,0.000088,0.000078,0.000030,0.000220,0.000057,0.000098
C38U.SI,0.000059,0.000057,0.000045,0.000060,0.000056,0.000060,0.000072,0.000018,0.000057,0.000183,0.000072


In [92]:
def calculate_volatility(weights, returns_covariance):
    cov_dot_weights = np.dot(returns_covariance * 252, weights)
    variance = np.dot(weights, cov_dot_weights)
    return np.sqrt(variance)


In [93]:
volatility = calculate_volatility(stock_weights, log_returns_cov)
print(volatility)

0.33418803915695855


### TO DO CONVERT NORMAL STANDARD DEVIATION AND NORMAL SHARPE RATIO

In [150]:
risk_free_rate = 0

# Calculate the expected portfolio return
log_portfolio_return = calculate_returns(stock_weights, log_returns)
portfolio_return = np.exp(log_portfolio_return)-1

# Calculate the portfolio volatility
portfolio_volatility = calculate_volatility(stock_weights, log_returns_cov)

# Calculate the Sharpe ratio
sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility

print("Portfolio Return:", portfolio_return)
print("Portfolio Volatility:", portfolio_volatility)
print("Sharpe Ratio:", sharpe_ratio)


Portfolio Return: 0.4225093215225195
Portfolio Volatility: 0.33418803915695855
Sharpe Ratio: 1.264286186269159


### Post Processing Weights using DiscreteAllocation Library

In [148]:
portfolio_value = 1000000

# Calculate the number of shares to purchase using DiscreteAllocation
#latest_prices = get_latest_prices(df)
earliest_prices = df.iloc[0]

da = DiscreteAllocation(cleaned_weights, earliest_prices, total_portfolio_value=portfolio_value)
allocation, leftover = da.lp_portfolio(solver="ECOS_BB")

print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

Discrete allocation: {'D05.SI': 31270, 'Z74.SI': 9839, 'J36.SI': 78, 'O39.SI': 2172, 'U11.SI': 1804, 'F34.SI': 57993, 'Y92.SI': 8388608, 'Q0F.SI': 100706, 'BN4.SI': 1368, 'C38U.SI': 125654, 'C07.SI': 140}
Funds remaining: $8631.34


In [120]:
stock_shares_allocation = list(allocation.values())
print(stock_shares_allocation)

[31270, 9839, 78, 2172, 1804, 57993, 8388608, 100706, 1368, 125654, 140]


In [140]:
# Calculate the daily portfolio value
daily_portfolio_value = np.dot(df, stock_shares_allocation)
daily_portfolio_value += leftover
df_portfolio_value = pd.DataFrame(data={'Portfolio Value': daily_portfolio_value}, index=df.index)
portfolio_log_returns = np.log(df_portfolio_value/df_portfolio_value.shift(1))

# Annual Returns
portfolio_annual_returns = portfolio_log_returns.mean() * 252
annual_regular_ret = np.exp(portfolio_annual_returns)-1


# Standard Deviation
daily_portfolio_sd = (np.exp(portfolio_log_returns)-1).std()
annual_std =  daily_portfolio_sd * (252 **0.5)

# Sharpe Ratio
sharpe_ratio_discrete = (annual_regular_ret - risk_free_rate) / annual_std

print("Total Portfolio Return with Discrete Allocation:", annual_regular_ret)
print("Portfolio Volatility with Discrete Allocation:", annual_std)
print("Sharpe Ratio with Discrete Allocation:", sharpe_ratio_discrete)
# Calculate the daily portfolio return series
#daily_portfolio_return_series = daily_portfolio_value.pct_change().dropna()


Total Portfolio Return with Discrete Allocation: Portfolio Value    0.283593
dtype: float64
Portfolio Volatility with Discrete Allocation: Portfolio Value    0.354116
dtype: float64
Sharpe Ratio with Discrete Allocation: Portfolio Value    0.800848
dtype: float64


'\n# Calculate the total portfolio return\ntotal_portfolio_return = np.sum(daily_portfolio_return_series)\n\n# Calculate the portfolio standard deviation\nportfolio_volatility_discrete = np.std(daily_portfolio_return_series)\n\n# Calculate the Sharpe ratio with discrete allocation\nsharpe_ratio_discrete = (total_portfolio_return - risk_free_rate) / portfolio_volatility_discrete\n\nprint("Total Portfolio Return with Discrete Allocation:", total_portfolio_return)\nprint("Portfolio Volatility with Discrete Allocation:", portfolio_volatility_discrete)\nprint("Sharpe Ratio with Discrete Allocation:", sharpe_ratio_discrete)\n'

In [149]:
df_portfolio_value

,Portfolio Value
Date,
2015-01-02 00:00:00+08:00,1.000000e+06
2015-01-05 00:00:00+08:00,9.864554e+05
2015-01-06 00:00:00+08:00,9.736864e+05
2015-01-07 00:00:00+08:00,9.780840e+05
2015-01-08 00:00:00+08:00,9.868572e+05
...,...
2022-12-23 00:00:00+08:00,7.317905e+06
2022-12-27 00:00:00+08:00,7.326558e+06
2022-12-28 00:00:00+08:00,7.365594e+06


### Variation 1

### Sector Constraints

In [146]:
sector_mapper = {
    "D05.SI": "Finance",
    "Z74.SI": "Telecommunication",
    "J36.SI": "Industrials",
    "O39.SI": "Finance",
    "U11.SI": "Finance",
    "F34.SI": "Financials",
    "Y92.SI": "Consumer Defensive",
    "Q0F.SI": "Healthcare",
    "BN4.SI": "Industrials",
    "C38U.SI": "Real Estate",
    "C07.SI": "Industrials"
}
sector_lower = {"Industrials": 0.1, "Telecommunication": 0.1, "Healthcare": 0.05} 
sector_upper = {
    "Consumer Defensive": 0.3,
    "Finance": 0.4
}

In [147]:
mu1 = mean_historical_return(df)
S1 = CovarianceShrinkage(df).ledoit_wolf()

ef1 = EfficientFrontier(mu1, S1)

ef1.add_sector_constraints(sector_mapper, sector_lower=sector_lower, sector_upper=sector_upper)

weights = ef1.max_sharpe()

cleaned_weights1 = ef1.clean_weights()
print(cleaned_weights1)

ef1.portfolio_performance(verbose=True)


OrderedDict([('D05.SI', 0.32924), ('Z74.SI', 0.1), ('J36.SI', 0.0744), ('O39.SI', 0.01132), ('U11.SI', 0.02534), ('F34.SI', 0.0841), ('Y92.SI', 0.3), ('Q0F.SI', 0.05), ('BN4.SI', 0.0256), ('C38U.SI', 0.0), ('C07.SI', 0.0)])
Expected annual return: 34.7%
Annual volatility: 31.3%
Sharpe Ratio: 1.05


(0.3471274623414157, 0.31272278117894203, 1.046062142029337)